# Step 2. KB(Knowledge Base) 검색

### Setup

In [ ]:
# 패키지 설치 후 Kernel Restart 를 수행하시고 다음 쉘로 진행하세요.

%pip install --upgrade pip
%pip install boto3 --force-reinstall --quiet
%pip install botocore --force-reinstall --quiet
%pip install sqlalchemy==2.0.0 --quiet
%pip install langchain --force-reinstall --quiet

### 사전에 생성한 KB 목록 확인

* 사용하려는 KB의 ID 를 확인합니다.

In [ ]:
import boto3

def list_knowledge_bases():
    # Bedrock 클라이언트 생성
    bedrock_agent = boto3.client('bedrock-agent')

    # Knowledge Base 목록 가져오기
    response = bedrock_agent.list_knowledge_bases(
        maxResults=100  # 한 번에 가져올 최대 결과 수
    )

    # 결과 출력
    if 'knowledgeBaseSummaries' in response:
        print("Found Knowledge Bases:")
        for kb in response['knowledgeBaseSummaries']:
            print(f"- Name: {kb['name']}")
            print(f"  ID: {kb['knowledgeBaseId']}")
            print(f"  Description: {kb.get('description', 'N/A')}")
            print(f"  Status: {kb['status']}")
            print(f"  Last Updated: {kb['updatedAt']}")
            print("---")
    else:
        print("No Knowledge Bases found.")

    # 페이지네이션 처리
    while 'nextToken' in response:
        response = bedrock_agent.list_knowledge_bases(
            maxResults=100,
            nextToken=response['nextToken']
        )
        
        for kb in response['knowledgeBaseSummaries']:
            print(f"- Name: {kb['name']}")
            print(f"  ID: {kb['knowledgeBaseId']}")
            print(f"  Description: {kb.get('description', 'N/A')}")
            print(f"  Status: {kb['status']}")
            print(f"  Last Updated: {kb['updatedAt']}")
            print("---")

if __name__ == "__main__":
    list_knowledge_bases()

### Bedrock Client 초기화

In [3]:
import boto3
import pprint
from botocore.client import Config
import json

pp = pprint.PrettyPrinter(indent=2)
session = boto3.session.Session()
region = session.region_name
bedrock_config = Config(connect_timeout=120, read_timeout=120, retries={'max_attempts': 0})
bedrock_client = boto3.client('bedrock-runtime', region_name = region)
bedrock_agent_client = boto3.client("bedrock-agent-runtime",
                              config=bedrock_config, region_name = region)
print(region)

us-west-2


In [4]:
# 사용자 쿼리

query = """
664 코드 알려줘?
"""

### KB 리트리버 설정 (Hybrid Search)
* 위에서 사용할 KB ID 를 확인하고 kb_id 에 수정하세요.

In [ ]:
# 위에서 KB ID를 확인하고 수정합니다.
kb_id = "<Enter KB ID>"

# numberOfResults 는 RAG 검색에서 가져올 청크 갯수 입니다.
def retrieve(query, kbId, numberOfResults=10):
    return bedrock_agent_client.retrieve(
        retrievalQuery= {
            'text': query
        },
        knowledgeBaseId=kbId,
        retrievalConfiguration= {
            'vectorSearchConfiguration': {
                'numberOfResults': numberOfResults,
                'overrideSearchType': "HYBRID", # optional
            }
        }
    )

# fetch context from the response
def get_contexts(retrievalResults):
    contexts = []
    for retrievedResult in retrievalResults: 
        contexts.append(retrievedResult['content']['text'])
    return contexts

response = retrieve(query, kb_id, 10)
retrievalResults = response['retrievalResults']
contexts = get_contexts(retrievalResults)

pp.pprint(contexts)

In [6]:
# 프롬프트 템플릿

prompt = f"""
Human: You are an advisor AI system, and provides answers to questions by using fact based when possible. 
Use the following pieces of information to provide a detail answer to the question enclosed in <question> tags. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
<context>
{contexts}
</context>

<question>
{query}
</question>

The response should be specific and use statistics or numbers when possible.

Assistant:"""

### Amazon Bedrock에서 Claude 3 모델 호출

In [7]:
# payload with model paramters
messages=[{ "role":'user', "content":[{'type':'text','text': prompt.format(contexts, query)}]}]
sonnet_payload = json.dumps({
    "anthropic_version": "bedrock-2023-05-31",
    "max_tokens": 4096,
    "messages": messages,
    "temperature": 0.0,
    "top_p": 0
        }  )

modelId = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"
# #modelId = "anthropic.claude-3-5-sonnet-20240620-v1:0"  # Claude 3.5 Sonnet
# modelId = 'anthropic.claude-3-sonnet-20240229-v1:0' # Claude 3 Sonnet
accept = 'application/json'
contentType = 'application/json'
response = bedrock_client.invoke_model(body=sonnet_payload, modelId=modelId, accept=accept, contentType=contentType)
response_body = json.loads(response.get('body').read())
response_text = response_body.get('content')[0]['text']

#pp.pprint(response_text)

print(response_text)

664 코드는 "Overlay fs Enabled Error"로, A/B 업데이트 중에 Remount가 일어난 경우에 발생하는 오류입니다.

영문 설명으로는 "During A/B update, Overlayfs is enabled (adb remount)"라고 되어 있습니다.

이 오류는 A/B 업데이트 과정에서 adb remount 명령어 등을 통해 파일 시스템이 overlayfs 모드로 마운트되었을 때 발생합니다. 이는 일반적으로 개발 목적으로 파일 시스템을 수정 가능한 상태로 만들었을 때 업데이트 과정에 영향을 줄 수 있어 발생하는 오류입니다.


---

---